In [ ]:
!pip install transformers
!pip install -U -q PyDrive

In [ ]:
!pip install simpletransformers
!pip freeze | grep simpletransformers

Tamil Data

In [3]:
import pandas as pd
df=pd.read_csv('/content/tam_3_train.tsv', delimiter='\t') 
df=df[["text", "category"]]
df = df.dropna()
df_dev=pd.read_csv('/content/tam_3_dev.tsv', delimiter='\t') 
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.dropna()
df_test=pd.read_csv('/content/tam_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'text'}) 
df_test = df_test.dropna()

In [4]:
tedf=pd.read_csv('/content/eng_tam_train.tsv', delimiter='\t') 
tedf=tedf[["text", "category"]]
tedf = tedf.dropna()
tedf_dev=pd.read_csv('/content/eng_tam_dev.tsv', delimiter='\t') 
tedf_dev=tedf_dev[["text", "category"]]
tedf_dev = tedf_dev.dropna()
tedf_test=pd.read_csv('/content/eng_tam_test.tsv', delimiter='\t') 
tedf_test = tedf_test.dropna()

### Mixing english and tamil translated to tamil data

In [5]:
train = pd.concat([df, tedf])
valid = pd.concat([df_dev,tedf_dev])
test = pd.concat([tedf_test,df_test])

In [6]:
print(len(train))
print(len(valid))
print(len(test))

5817
1456
991


In [7]:
concatenated = pd.concat([train, test,valid])

import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (4958, 2)
Validation set shape: (1653, 2)
Test set shape: (1653, 2)


In [8]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

Non-anti-LGBT+ content    4319
Homophobic                 518
Transphobic                121
Name: category, dtype: int64
Non-anti-LGBT+ content    1440
Homophobic                 173
Transphobic                 40
Name: category, dtype: int64
Non-anti-LGBT+ content    1440
Homophobic                 173
Transphobic                 40
Name: category, dtype: int64


In [9]:
train_df['category'] = train_df['category'].replace('Non-anti-LGBT+ content', 'ally')
val_df['category'] = val_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')

In [10]:
train_df['category'] = train_df['category'].replace('ally', 0)
train_df['category'] = train_df['category'].replace('Homophobic', 1)
train_df['category'] = train_df['category'].replace('Transphobic', 2)

In [11]:
val_df['category'] = val_df['category'].replace('ally', 0)
val_df['category'] = val_df['category'].replace('Homophobic', 1)
val_df['category'] = val_df['category'].replace('Transphobic', 2)

In [12]:
test_df['category'] = test_df['category'].replace('ally', 0)
test_df['category'] = test_df['category'].replace('Homophobic', 1)
test_df['category'] = test_df['category'].replace('Transphobic', 2)

In [13]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [14]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=8a58d021552a2aeae9eb2612dc32141f2bb408df451699246baadc5f26dd789e
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [15]:
import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Training Model with Englis Data

In [16]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 16,
             "eval_batch_size": 16,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 2e-5}

# Create a ClassificationModel
model = ClassificationModel(
    "xlmroberta", "xlm-roberta-base",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:486: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [17]:
# Train the model
model.train_model(train_df)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/310 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/310 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/310 [00:00<?, ?it/s]

(930, 0.36625689045395904)

Validation Set


In [18]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1653 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

{'mcc': 0.5234826225571126,
 'f1': 0.5079202283646347,
 'acc': 0.8983666061705989,
 'eval_loss': 0.29881963580667686}

Testing With Tamil Data

In [19]:
test_df

,text,labels
1173,அவள் அழகாக இருக்கிறாள்,0
790,MyNameIsRam நன்றி bro red_heart,0
2410,உரக்கச் சொல் உன் உண்மையே ...........கேட்குரவன்...,0
470,"சிஸ்ஸி, உன்னிடம் சிறந்த மனப்பான்மை, நம்பிக்கை ...",1
362,சங்கிகளை எதிர்க்கும் குயர்கள் red_heart fire,0
...,...,...
739,எனது கருத்தை விரும்பு,0
923,"முதலில், நான் ஒன்று தெரிந்து கொள்ள விரும்புகிற...",1
156,கலாச்சார சீர்கேடு ஏற்பட வாய்ப்பு உள்ளது..........,1
1269,அருமையான பேச்சு மாலினி..... லவ் யூ லாட்.... நா...,0


In [20]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1653 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"சிஸ்ஸி, உன்னிடம் சிறந்த மனப்பான்மை, நம்பிக்கை ...",1,0
1,யாரெல்லாம் கடவுள் ஆணுக்கு பெண்ண படச்சார் -னு ச...,0,1
2,அட அரிபெடுத்த straight நாய்களா... நம்ம நாட்டோட...,0,1
3,டை தேவுடியா நாயா எங்க நாட்டை வீட்டு செல்,1,0
4,இந்தமாதிரி சில திருநங்கை நாய்களால்தான் மற்றவர்...,2,1
...,...,...,...
161,இது வரவேற்கத்தக்க தீர்ப்பு. ஏனென்றால் தன்பாலின...,0,1
162,சுத்தி நிக்கற அத்தன பேரும் சோ்ந்தா இதுகள மெரட...,0,1
163,Anchor ku பேச தெரியல ....,1,0
164,"நான் ஓரினச்சேர்க்கையில் உடன்படவில்லை, ஏனென்றால...",1,0


In [21]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.94      0.96      0.95      1440
  homophobic       0.57      0.58      0.58       173
 transphobic       0.00      0.00      0.00        40

    accuracy                           0.90      1653
   macro avg       0.50      0.51      0.51      1653
weighted avg       0.88      0.90      0.89      1653



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with Tamil and testing with english

In [22]:
o_test=pd.read_csv('/content/tam_eng_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
e_test = e_test.rename(columns={'text                        ': 'text'})
#e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [23]:
test_ml = pd.concat([o_test,e_test])
test_ml

,Unnamed: 0,text,category,ln
0,0.0,When each commanta was read and brought up.,Non-anti-LGBT+ content,tam_eng
1,1.0,Bro live a prosperous bouquet OK_hand OK_hand ...,Non-anti-LGBT+ content,tam_eng
2,2.0,Jesus is a step sister,Non-anti-LGBT+ content,tam_eng
3,3.0,Eyes are confused....,Non-anti-LGBT+ content,tam_eng
4,4.0,Come together Gay mean not sex it's love also ...,Non-anti-LGBT+ content,tam_eng
...,...,...,...,...
985,NaN,Looks like Karthik took advantage on Varun whe...,Non-anti-LGBT+ content,NaN
986,NaN,i am really crying pro😢😢😢😭😭😭😭😭😭 ...,Non-anti-LGBT+ content,NaN
987,NaN,They may be transgender but don't ever forgot ...,Non-anti-LGBT+ content,NaN
988,NaN,It is their own choice. I support them No doub...,Non-anti-LGBT+ content,NaN


In [24]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [25]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [26]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [27]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"Similarly, in all matters, the law will come a...",1,0
1,Gay ah they are not wrong...it is created by L...,0,1
2,pranesh G you are a transgender.,2,0
3,Do the people in your house agree to this type...,1,0
4,Then do not eat by mouth,1,0
...,...,...,...
204,Common people cannot understand the DNA abbera...,0,1
205,"First of all, i wanna know onething. Do you th...",1,0
206,madam nalini....u talking too much right....ok...,1,0
207,Friendship s friendship.dont make it as sexual...,1,0


In [28]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.91      0.97      0.94      1579
  homophobic       0.60      0.38      0.46       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.89      1819
   macro avg       0.50      0.45      0.47      1819
weighted avg       0.85      0.89      0.86      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with Tamil and testing with Hindi

In [29]:
o_test=pd.read_csv('/content/eng_hin_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_hin_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [30]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [31]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [32]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [33]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [34]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"आप दोनों जिन्होंने मेरी टिप्पणी का जवाब दिया, ...",1,0
1,मुझे लगता है..यह मानसिक बीमारी है। जाओ और डॉक्...,1,0
2,पुलिस को उन्हें सजा देनी चाहिए,1,0
3,निर्देशक सर pls गे रिपेंट अगुरा मारी फिल्म एडुंगा,1,0
4,यह जानवरों और कई अन्य प्रजातियों में भी है। आप...,0,1
...,...,...,...
223,वह बहुत अच्छा काम करती है। मांचूर में डेढ़ साल...,2,0
224,मैं आपकी राय मानता हूं। अगर समलैंगिकता गलत है ...,1,0
225,इसी महीने कुत्तों को देखते ही गोली मार दी गई,2,1
226,सुंदर वाणी मधुर वाणी। सवाल का जवाब बहुत अच्छा ...,1,0


In [35]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.91      0.97      0.93      1579
  homophobic       0.49      0.34      0.40       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.46      0.44      0.45      1819
weighted avg       0.84      0.87      0.85      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with Tamil and testing with Telugu

In [36]:
o_test=pd.read_csv('/content/eng_tel_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_tel_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [37]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [38]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [39]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [40]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [41]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"సంస్కృతి గురించి మాట్లాడే వారికి, వివాహానికి మ...",0,1
1,మీరు గంజాయి మరియు THC 'tetrahydrocannabinol సమ...,0,1
2,నా వ్యాఖ్యకు ప్రత్యుత్తరం ఇచ్చిన మీరిద్దరూ దీన...,1,0
3,నా అభిప్రాయం .. ఇది మానసిక వ్యాధి. వెళ్లి వైద్...,1,0
4,పోలీసులు వారిని శిక్షించాలి,1,0
...,...,...,...
228,ఆమె చాలా చక్కగా నటిస్తుంది. మంచూర్‌లో డబ్బులు ...,2,0
229,నేను మీ అభిప్రాయాన్ని అంగీకరిస్తున్నాను. స్వలి...,1,0
230,ఈ నెలలో కుక్కలను చూసి కాల్చి చంపారు,2,1
231,చక్కని ప్రసంగం. మధురమైన స్వరం. అన్న ప్రశ్నకు స...,1,0


In [42]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.90      0.97      0.93      1579
  homophobic       0.48      0.30      0.37       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.46      0.42      0.43      1819
weighted avg       0.83      0.87      0.85      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training with Tamil and testing with Malyalam

In [43]:
o_test=pd.read_csv('/content/eng_ml_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_ml_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [44]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [45]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [46]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [47]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [48]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"സംസ്കാരത്തെക്കുറിച്ച് സംസാരിക്കുന്നവർക്ക്, വിവ...",0,1
1,കഞ്ചാവിനെക്കുറിച്ചും THC 'tetrahydrocannabinol...,0,1
2,എന്റെ അഭിപ്രായത്തിന് മറുപടി നൽകിയ നിങ്ങൾ രണ്ടു...,1,0
3,എനിക്ക് തോന്നുന്നു .. മാനസിക രോഗമാണ്. പോയി ഡോക...,1,0
4,പോലീസ് അവരെ ശിക്ഷിക്കണം,1,0
...,...,...,...
216,അവൾ വളരെ ഭംഗിയായി അഭിനയിക്കുന്നു. മഞ്ചൂരിൽ സർക...,2,0
217,താങ്കളുടെ അഭിപ്രായം ഞാൻ അംഗീകരിക്കുന്നു. സ്വവർ...,1,0
218,ഈ മാസം നായ്ക്കളെ കണ്ടപ്പോൾ വെടിവച്ചു,2,1
219,നല്ല സംസാരം.മധുരമായ ശബ്ദം. എന്ന ചോദ്യത്തിനുള്ള...,1,0


In [49]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.91      0.97      0.94      1579
  homophobic       0.51      0.37      0.43       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.88      1819
   macro avg       0.47      0.44      0.45      1819
weighted avg       0.84      0.88      0.86      1819



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
